In [2]:
from tensorflow.keras.models import load_model
import IPython.display as ipd
import numpy as np

/Users/johannasmriti/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [18]:
autoencoder=load_model('/Users/johannasmriti/Downloads/Final working model/denoiser/denoisingAudio5sec2.h5')

In [19]:
class MeanVarianceNorm():
    def fit(self, data_to_fit):
        self.mean = np.mean(data_to_fit)
        self.std = np.std(data_to_fit)
    def normalize(self, data_to_transform):
        return (data_to_transform - self.mean) / self.std
    def denormalize(self, data_to_transform):
        return (data_to_transform * self.std) + self.mean

In [ ]:
import numpy as np
import librosa
import soundfile as sf
import cv2

def predict_denoise_audio(noisy_audio_filepath, sample_rate=16000, duration_s=5):
    data, _ = librosa.load(noisy_audio_filepath, sr=sample_rate)
    duration = int(sample_rate * duration_s)
    
    if len(data) < duration:
        data = np.pad(data, (0, duration - len(data)), 'constant')  
    elif len(data) > duration:
        data = data[:duration]  
    S_noisy = np.abs(librosa.stft(data, n_fft=2048))[:-1, :]
    
    S_noisy_resized = cv2.resize(S_noisy, (173, 1024)).T  
    
    N = MeanVarianceNorm()
    N.fit(S_noisy_resized)  
    S_noisy_n = N.normalize(S_noisy_resized)
    
    
    S_noisy_n = np.expand_dims(S_noisy_n.T, axis=-1)  
    S_noisy_n = np.expand_dims(S_noisy_n, axis=0)  
    
    predicted_S_clean_n = autoencoder.predict(S_noisy_n)
    
    predicted_S_clean = N.denormalize(predicted_S_clean_n[0])
    
    print(f"Predicted Spectrogram Shape: {predicted_S_clean.shape}")
    print(f"Predicted Spectrogram Values: {predicted_S_clean[:5]}")
    
    S_noisy_phase = np.angle(librosa.stft(data, n_fft=2048))[:-1, :]
    
    S_noisy_phase_resized = cv2.resize(S_noisy_phase, (173, S_noisy_phase.shape[0])).T  
    
    S_noisy_phase_resized = S_noisy_phase_resized.T  
    
    S_clean_complex = predicted_S_clean.squeeze() * np.exp(1j * S_noisy_phase_resized)
    
    data_clean_inv = librosa.istft(S_clean_complex)
    
    if np.any(np.isnan(data_clean_inv)):
        print("Warning: Cleaned audio contains NaN values.")
        return None
    if data_clean_inv.size == 0:
        print("Error: Cleaned audio is empty.")
        return None
    
    sf.write('cleaned_audio.wav', data_clean_inv, sample_rate)
    
    
    return 'cleaned_audio.wav'

noisy_audio_filepath = '/Users/johannasmriti/Downloads/Final working model/denoiser/noisy_example.wav'
predicted_audio = predict_denoise_audio(noisy_audio_filepath)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
Predicted Spectrogram Shape: (1024, 173, 1)
Predicted Spectrogram Values: [[[0.48827326]
  [0.5385084 ]
  [0.54499036]
  [0.54445165]
  [0.5754477 ]
  [0.5573375 ]
  [0.6145939 ]
  [0.61949146]
  [0.5542169 ]
  [0.48801988]
  [0.488376  ]
  [0.48771477]
  [0.50992227]
  [0.45970878]
  [0.42681384]
  [0.42956442]
  [0.42088658]
  [0.43957362]
  [0.4336178 ]
  [0.42948133]
  [0.47261778]
  [0.4555321 ]
  [0.45722216]
  [0.434875  ]
  [0.37542406]
  [0.382062  ]
  [0.3660467 ]
  [0.3681812 ]
  [0.3631523 ]
  [0.3671157 ]
  [0.37342206]
  [0.35309026]
  [0.375843  ]
  [0.3820594 ]
  [0.38864356]
  [0.4029808 ]
  [0.41366315]
  [0.40251154]
  [0.38710445]
  [0.37545177]
  [0.39307365]
  [0.40918124]
  [0.3959294 ]
  [0.39950693]
  [0.3957676 ]
  [0.3824878 ]
  [0.37428555]
  [0.37046862]
  [0.36590603]
  [0.37520465]
  [0.39066496]
  [0.40746897]
  [0.4263418 ]
  [0.4360031 ]
  [0.4338496 ]
  [0.4325502 ]
  [0.40542   ]
  [0.3851381 ]
  [0.38773677]
  

In [21]:
ipd.Audio('cleaned_audio.wav')

In [22]:
ipd.Audio(noisy_audio_filepath)